Speckle Reducing Anisotropic Diffusion, Yongjian Yu and Scott T. Acton, Senior Member, IEEE

In [1]:
import numpy as np
from skimage import io
from scipy import signal
from scipy.spatial.distance import cdist
from scipy.linalg import norm

In [2]:
EPSILON = 0.001

def laplacian(image: np.array) -> np.array: 
    """Eq. (15)"""
    len_y, len_x = image.shape
    image_new = np.zeros([len_y, len_x], dtype=np.float32)
    for j in range(1, len_y-1):
        for i in range(1, len_x-1):
            image_new[j,i] = (image[j-1,i] + image[j+1,i] + image[j,i-1] + image[j,i+1]) - 4 * image[j,i]
    return image_new


def SRAD(image: np.array, rho: float, n_diffusion: int) -> np.array:
    """Anisotropic Diffusion calculation"""
    q0 = 1 # under Eq. (37)
    for t in range(1, n_diffusion):
        len_y = image.shape[0]
        len_x = image.shape[1]
        
        I = np.zeros([len_y+2, len_x+2], dtype=np.float32)
        I[1:len_y+1, 1:len_x+1] = image
        
        q0 = q0 * np.exp(-t*rho) # Eq. (37)
        DL_x = signal.convolve2d(I, [[0,0,0],[0,1,-1],[0,0,0]], mode='same', boundary='symm') # Eq. (28)
        DL_y = signal.convolve2d(I, [[0,-1,0],[0,1,0],[0,0,0]], mode='same', boundary='symm') # Eq. (28)
        DR_x = signal.convolve2d(I, [[0,0,0],[1,-1,0],[0,0,0]], mode='same', boundary='symm') # Eq. (29)
        DR_y = signal.convolve2d(I, [[0,0,0],[0,-1,0],[0,1,0]], mode='same', boundary='symm') # Eq. (29)

        D_I =  np.sqrt(DL_x**2 + DL_y**2 + DR_x**2 + DR_y**2) / np.sqrt(2) # under Eq. (29)
        q = np.sqrt((1/2 * D_I**2 + 1/16 * laplacian(I)**2) / (I + 1/4 * laplacian(I) + EPSILON)**2)  # Eq. (31)
        
        c = 1 / (1 + ((q**2 - q0**2)/(q0**2 * (1 + q0**2)))) # Eq. (33)
        d_t = signal.convolve2d(c, [[0,0,0],[0,0,-1],[0,0,0]],  mode='same', boundary='symm') * \
              signal.convolve2d(I, [[0,0,0],[0,1,-1],[0,0,0]], mode='same', boundary='symm') +  \
              signal.convolve2d(c, [[0,0,0],[0,-1,0],[0,0,0]],  mode='same', boundary='symm') * \
              signal.convolve2d(I, [[0,0,0],[-1,1,0],[0,0,0]], mode='same', boundary='symm') +  \
              signal.convolve2d(c, [[0,-1,0],[0,0,0],[0,0,0]],  mode='same', boundary='symm') * \
              signal.convolve2d(I, [[0,-1,0],[0,1,0],[0,0,0]], mode='same', boundary='symm') +  \
              signal.convolve2d(c, [[0,0,0],[0,-1,0],[0,0,0]],  mode='same', boundary='symm') * \
              signal.convolve2d(I, [[0,0,0],[0,1,0],[0,-1,0]], mode='same', boundary='symm')
        I = I + rho/4 * d_t  # Eq. (61)
        image = I[1:len_y+1, 1:len_x+1]
    return image

In [3]:
class fuzzy_cluster: # see https://www.geeksforgeeks.org/ml-fuzzy-clustering/
    def __init__(self, n_clusters=10, max_iter=100, m=2, error=1e-5, random_state=42):
        self.gamma = None
        self.centers = None
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.m = m #  fuzziness parameter 
        self.error = error
        self.random_state = random_state

    def fit(self, image_vec):
        n_pixels = image_vec.shape[0]
        C = self.n_clusters
        centers = []
        
        r = np.random.RandomState(self.random_state)
        gamma = r.rand(n_pixels, C) # init the cluster memberships of each point in image_vec
        gamma = gamma / np.tile(gamma.sum(axis=1)[np.newaxis].T, C) # normalizing the memberships 
        
        for i in range(self.max_iter):
            temp_gamma = gamma.copy()
            centers = self.new_centers(image_vec, gamma)
            gamma = self.new_gamma(image_vec, centers)
            if norm(gamma - temp_gamma) < self.error:
                break
        
        self.gamma = gamma
        self.centers = centers
        return self

    def new_centers(self, image_vec, gamma):
        """find out the centroid"""
        gamma_m = gamma ** self.m
        return (image_vec.T @ gamma_m / np.sum(gamma_m, axis=0)).T

    def new_gamma(self, image_vec, centers):
        """updating membership values"""
        temp = cdist(image_vec, centers) ** self.m
        denominator_ = temp.reshape((image_vec.shape[0], 1, -1)).repeat(temp.shape[-1], axis=1)
        denominator_ = temp[:, :, np.newaxis] / denominator_
        
        return 1 / denominator_.sum(2) ** (1 / (self.m - 1))


In [13]:
def hierarchical_clustering(image, second_cluster_size=5, precentile_h=1.25 , precentile_l=0.9):
    """ preclassification by hierarchical clustering.
        pixels with high probability to be changed are labeled 2.
        pixels with uncertainty are labeled 1.
        pixels with high probability to be unchanged are labeled 0.
       """
    len_y, len_x = image.shape
    pix_vec = image.reshape([len_y * len_x, 1])
    
    # 1st clustering#
    fcm = fuzzy_cluster(n_clusters=2)
    fcm.fit(pix_vec)
    fcm_lab = fcm.gamma.argmax(axis=1)
   
    if sum(fcm_lab==0)<sum(fcm_lab==1):
        ttr = round(sum(fcm_lab==0) * precentile_h)
        ttl = round(sum(fcm_lab==0) * precentile_l)
    else:
        ttr = round(sum(fcm_lab==1) * precentile_h)
        ttl = round(sum(fcm_lab==1) * precentile_l)
        
     # 2nd clustering#
    fcm = fuzzy_cluster(n_clusters=second_cluster_size)
    fcm.fit(pix_vec)
    fcm_lab  = fcm.gamma.argmax(axis=1)
    idx = []
    idx_tmp = []
    idxmean = []
    res_lab = np.zeros(len_y*len_x, dtype=np.float32)
    for i in range(0, second_cluster_size):
        idx_tmp.append(np.argwhere(fcm_lab==i))
        idxmean.append(image.reshape(len_y*len_x, 1)[idx_tmp[i]].mean())
    
    idx_sort = np.argsort(idxmean)
    for i in range(0, second_cluster_size):
        idx.append(idx_tmp[idx_sort[i]])
    
    loc = second_cluster_size - 1
    c = len(idx[loc])
    res_lab[idx[loc]] = 2
    flag_mid = 0
    for i in range(1, second_cluster_size):
        c = c + len(idx[loc-i])
        
        if c < ttl:
            res_lab[idx[loc-i]] = 2
        elif c >= ttl and c < ttr:
            res_lab[idx[loc-i]] = 1
            flag_mid = 1
        elif flag_mid == 0:
            res_lab[idx[loc-i]] = 1
            flag_mid = 1
    res_lab = res_lab.reshape(len_y, len_x)
    return res_lab

In [5]:
im1_path  = 'ottawa_1.bmp'
im2_path  = 'ottawa_2.bmp'
rho = 0.15 # diffusion parameter
n_times = 6 # number of times performing diffusion 
im1 = io.imread(im1_path)[:,:,0].astype(np.float32)
im2 = io.imread(im2_path)[:,:,0].astype(np.float32)

In [9]:
im_log_diff = np.log(SRAD(im1, rho, n_times) + 1) - np.log(SRAD(im2, rho, n_times) + 1) # calculating logarithmic difference
im_diff = SRAD(im_log_diff, rho, n_times)

In [7]:
# pixels with high probability to be changed are labeled 2.
# pixels with uncertainty are labeled 1.
# pixels with high probability to be unchanged are labeled 0.
hierarchical_clustering(im_diff)

starting 1st clustering
starting 2nd clustering


array([[1., 1., 0., ..., 0., 2., 2.],
       [1., 1., 0., ..., 0., 1., 2.],
       [1., 0., 0., ..., 0., 1., 1.],
       ...,
       [1., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 1., 1., 1.],
       [1., 2., 0., ..., 2., 2., 2.]], dtype=float32)